# Python для анализа данных

## Что такое SQL. Как писать запросы. Работа с Clickhouse. 

Автор: *Ян Пиле, НИУ ВШЭ*

Язык SQL очень прочно влился в жизнь аналитиков и требования к кандидатам благодаря простоте, удобству и распространенности. Часто SQL используется для формирования выгрузок, витрин (с последующим построением отчетов на основе этих витрин) и администрирования баз данных. Поскольку повседневная работа аналитика неизбежно связана с выгрузками данных и витринами, навык написания SQL запросов может стать крайне полезным умением. 

Рассказ о том, что такое базы данных и в каком виде данные хранятся, требуется отдельный – тема очень большая. Мы будем представлять себе базу данных как набор хранящихся на нескольких серверах таблиц с именованными колонками (как в Excel-файлах, только больше (на самом деле это не совсем так, но нам усложнение логики не потребуется)

#### Структура sql-запросов

Общая структура запроса выглядит следующим образом:

* SELECT ('столбцы или * для выбора всех столбцов; обязательно')
* FROM ('таблица; обязательно')
* WHERE ('условие/фильтрация, например, city = 'Moscow'; необязательно')
* GROUP BY ('столбец, по которому хотим сгруппировать данные; необязательно')
* HAVING ('условие/фильтрация на уровне сгруппированных данных; необязательно')
* ORDER BY ('столбец, по которому хотим отсортировать вывод; необязательно')
* LIMIT ('число, сколько строк результата надо вывести; необязательно')

Введение в синтаксис будет рассмотрено на примере открытой базы данных, предназначенной специально для практики SQL. Чтобы твое обучение прошло максимально эффективно, открой ссылку ниже в новой вкладке и сразу запускай приведенные примеры, это позволит тебе лучше закрепить материал и самостоятельно поработать с синтаксисом.

https://www.w3schools.com/sql/trysql.asp?filename=trysql_op_in

После перехода по ссылке можно будет увидеть сам редактор запросов и вывод данных в центральной части экрана, список таблиц базы данных находится в правой части.


Разберем структуру. Для удобства текущий изучаемый элемент в запроса выделяется CAPS'ом.

SELECT, FROM

SELECT, FROM — обязательные элементы запроса, которые определяют выбранные столбцы, их порядок и источник данных.

Выбрать все (обозначается как *) из таблицы Customers:

**SELECT * FROM Customers**

Выбрать столбцы CustomerID, CustomerName из таблицы Customers:

**SELECT CustomerID, CustomerName FROM Customers**

WHERE

WHERE — необязательный элемент запроса, который используется, когда нужно отфильтровать данные по нужному условию. Очень часто внутри элемента where используются IN / NOT IN для фильтрации столбца по нескольким значениям, AND / OR для фильтрации таблицы по нескольким столбцам.

Фильтрация по одному условию и одному значению:

select * from Customers
WHERE City = 'London'

Фильтрация по одному условию и нескольким значениям с применением IN (включение) или NOT IN (исключение):

select * from Customers
where City IN ('London', 'Berlin')

select * from Customers
where City NOT IN ('Madrid', 'Berlin','Bern')

Фильтрация по нескольким условиям с применением AND (выполняются все условия) или OR (выполняется хотя бы одно условие) и нескольким значениям:

select * from Customers
where Country = 'Germany' AND City not in ('Berlin', 'Aachen') AND CustomerID > 15

select * from Customers
where City in ('London', 'Berlin') OR CustomerID > 4


GROUP BY

GROUP BY — необязательный элемент запроса, с помощью которого можно задать агрегацию по нужному столбцу (например, если нужно узнать какое количество клиентов живет в каждом из городов).

При использовании GROUP BY обязательно:

перечень столбцов, по которым делается разрез, был одинаковым внутри SELECT и внутри GROUP BY,
агрегатные функции (SUM, AVG, COUNT, MAX, MIN) должны быть также указаны внутри SELECT с указанием столбца, к которому такая функция применяется.

Группировка количества клиентов по городу:

select City, count(CustomerID) from Customers
GROUP BY City

Группировка количества клиентов по стране и городу:

select Country, City, count(CustomerID) from Customers
GROUP BY Country, City

Группировка продаж по ID товара с разными агрегатными функциями: количество заказов с данным товаром и количество проданных штук товара:


select ProductID, COUNT(OrderID), SUM(Quantity) from OrderDetails
GROUP BY ProductID

Группировка продаж с фильтрацией исходной таблицы. В данном случае на выходе будет таблица с количеством клиентов по городам Германии:


select City, count(CustomerID) from Customers
WHERE Country = 'Germany'
GROUP BY City

Переименование столбца с агрегацией с помощью оператора AS. По умолчанию название столбца с агрегацией равно примененной агрегатной функции, что далее может быть не очень удобно для восприятия.

select City, count(CustomerID) AS Number_of_clients from Customers
group by City

HAVING

HAVING — необязательный элемент запроса, который отвечает за фильтрацию на уровне сгруппированных данных (по сути, WHERE, но только на уровень выше).

Фильтрация агрегированной таблицы с количеством клиентов по городам, в данном случае оставляем в выгрузке только те города, в которых не менее 5 клиентов:


select City, count(CustomerID) from Customers
group by City
HAVING count(CustomerID) >= 5 


В случае с переименованным столбцом внутри HAVING можно указать как и саму агрегирующую конструкцию count(CustomerID), так и новое название столбца number_of_clients:


select City, count(CustomerID) as number_of_clients from Customers
group by City
HAVING number_of_clients >= 5

Пример запроса, содержащего WHERE и HAVING. В данном запросе сначала фильтруется исходная таблица по пользователям, рассчитывается количество клиентов по городам и остаются только те города, где количество клиентов не менее 5:


select City, count(CustomerID) as number_of_clients from Customers
WHERE CustomerName not in ('Around the Horn','Drachenblut Delikatessend')
group by City
HAVING number_of_clients >= 5

ORDER BY

ORDER BY — необязательный элемент запроса, который отвечает за сортировку таблицы.

Простой пример сортировки по одному столбцу. В данном запросе осуществляется сортировка по городу, который указал клиент:


select * from Customers
ORDER BY City

Осуществлять сортировку можно и по нескольким столбцам, в этом случае сортировка происходит по порядку указанных столбцов:


select * from Customers
ORDER BY Country, City

По умолчанию сортировка происходит по возрастанию для чисел и в алфавитном порядке для текстовых значений. Если нужна обратная сортировка, то в конструкции ORDER BY после названия столбца надо добавить DESC:


select * from Customers
order by CustomerID DESC

Обратная сортировка по одному столбцу и сортировка по умолчанию по второму:

select * from Customers
order by Country DESC, City

JOIN

JOIN — необязательный элемент, используется для объединения таблиц по ключу, который присутствует в обеих таблицах. Перед ключом ставится оператор ON.

Запрос, в котором соединяем таблицы Order и Customer по ключу CustomerID, при этом перед названиям столбца ключа добавляется название таблицы через точку:

select * from Orders
JOIN Customers ON Orders.CustomerID = Customers.CustomerID

Нередко может возникать ситуация, когда надо промэппить одну таблицу значениями из другой. В зависимости от задачи, могут использоваться разные типы присоединений. INNER JOIN — пересечение, RIGHT/LEFT JOIN для мэппинга одной таблицы знаениями из другой,


select * from Orders
join Customers on Orders.CustomerID = Customers.CustomerID
where Customers.CustomerID >10

Теперь попробуем порешать задачи в интерфейсе Clickhouse. Он называется Tabix.

Web-нтерфейс Clickhouse располагается по адресу: **Tabix.beslan.pro** 
Наша база данных Clickhouse состоит из четырех таблиц:
    
* **events** (события в приложении)
* **checks** (чеки покупок в приложении)
* **devices** (идентификаторы устройств, на которые приложения установлены)
* **installs** (установки приложений)

Возьмем одну из таблиц и опробуем элементы запроса на ней. Пусть это будет таблица events.
Для простоты визуализации результатов запроса мы будем загружать данные прямо в Python (для этого уже написана функция, а уже потом будем разбирать, как эта функция работает.

In [1]:
!pip install pandahouse

In [2]:
import json # Чтобы разбирать поля
import requests # чтобы отправлять запрос к базе
import pandas as pd # чтобы в табличном виде хранить результаты запроса

USER = 'student'
PASS = 'dpo_python_2020'
HOST = 'http://clickhouse.beslan.pro:8080/'

def get_clickhouse_data(query,
                        host=HOST, 
                        USER = USER, 
                        PASS = PASS, 
                        connection_timeout = 1500, 
                        dictify=True, 
                        **kwargs):
    NUMBER_OF_TRIES = 5  # Количество попыток запуска
    DELAY = 10           #время ожидания между запусками
    import time
    params = kwargs      #если вдруг нам нужно в функцию положить какие-то параметры
    if dictify:
        query += "\n FORMAT JSONEachRow"   # dictify = True отдает каждую строку в виде JSON'a

    for i in range(NUMBER_OF_TRIES):

#         headers = {'Accept-Encoding': 'gzip'}

        r = requests.post(host, 
                          params = params, 
                          auth=(USER, PASS), 
                          timeout = connection_timeout, 
                          data=query
                          )              # отправили запрос на сервер

        if r.status_code == 200 and not dictify:    
            return r.iter_lines()         # генератор :)
        elif r.status_code == 200 and dictify:
            return (json.loads(x) for x in r.iter_lines()) # генератор :)
        
        else:
            print('ATTENTION: try #%d failed' % i)
            if i != (NUMBER_OF_TRIES - 1):
                print(r.text)
                time.sleep(DELAY * (i + 1))
            else:
                raise(ValueError, r.text)
                
def get_data(query):
    return pd.DataFrame(list(get_clickhouse_data(query, dictify=True)))

SQL-запросы мы будем сгружать в функцию в виде текста. 

In [3]:
query = """
select  *
from events
limit 10
"""

In [4]:
gg = get_data(query)

In [5]:
gg

,AppPlatform,events,EventDate,DeviceID
0,android,8,2019-09-29,7429291373250434008
1,android,175,2019-09-15,7429291824672902510
2,android,0,2019-09-17,7429291824672902510
3,android,0,2019-09-26,7429291824672902510
4,android,4,2019-04-29,7429292273953361459
5,android,38,2019-08-20,7429293114537639018
6,android,38,2019-05-21,7429298825563999474
7,android,4,2019-05-26,7429298825563999474
8,android,100,2019-08-07,7429300397574411770
9,android,26,2019-01-31,7429301272237917347


Отлично! Мы достали 10 записей таблицы events. Теперь опробуем выражение where: достанем только те записи, которые соответствуют платформе iOS.

In [6]:
query = """
select  *
from events
where AppPlatform ='iOS'
limit 10
"""

In [7]:
get_data(query)

,AppPlatform,events,EventDate,DeviceID
0,iOS,12,2019-04-10,729846050352974830
1,iOS,26,2019-07-11,729846050352974830
2,iOS,24,2019-07-21,729846050352974830
3,iOS,26,2019-09-16,729846050352974830
4,iOS,20,2019-01-30,729854859919179786
5,iOS,7,2019-02-22,729854859919179786
6,iOS,2,2019-05-12,729854859919179786
7,iOS,8,2019-05-22,729854859919179786
8,iOS,24,2019-06-16,729854859919179786
9,iOS,113,2019-07-22,729854859919179786


Теперь остается попробовать выражения group by, having, order by и какую-нибудь группировочную функцию. Предлагаю посчитать количество событий (сумму поля events) в платформе iOS за июнь 2019 года, отсортировав выдачу по дате и выводя только дни, количество событий в которых было больше 6000000

In [8]:
query = """
select *
from (select EventDate, sum(events) as events_cnt
from events
where AppPlatform ='iOS'
    and EventDate between'2019-06-01' and '2019-06-30'
group by EventDate
having sum(events)>6000000
order by EventDate)
"""
get_data(query)

,EventDate,events_cnt
0,2019-06-03,6494328
1,2019-06-04,7460271
2,2019-06-05,6367228
3,2019-06-06,6701694
4,2019-06-07,6086262
5,2019-06-10,6426396
6,2019-06-11,7142591
7,2019-06-12,6397508
8,2019-06-13,6515225
9,2019-06-14,6305829


А еще существует понятие "подзапрос", имеется в виду, что вы в одном запросе обращаетесь к результатам другого запроса. Например можно посчитать количество дней, в которые events_cnt было больше 6000000

In [9]:
query = """
 select count() as days_cnt
 from   
     (select EventDate, sum(events) as events_cnt
    from events
    where AppPlatform ='iOS'
        and EventDate between'2019-06-01' and '2019-06-30'
    group by EventDate
    having sum(events)>6000000
    order by EventDate)
"""
get_data(query)

,days_cnt
0,13


Кроме того результаты подзапроса можно передать в блок where. Давайте попробуем достать те DeviceID, которые совершили более 1300 событий 2019-05-15

In [10]:
query = """
select DeviceID
from events
where EventDate = '2019-05-15' 
group by DeviceID
having sum(events)>=1300
"""
get_data(query)

,DeviceID
0,16143542820771817388
1,5139982497053240133
2,11853597032288080243


А теперь достанем количество событий, которые совершили эти DeviceID за июнь 2019 в разбивке по дням.

In [11]:
query = """
 select EventDate, sum(events) as events_cnt
from events
where EventDate between'2019-06-01' and '2019-06-30'
    and DeviceID in (select DeviceID
                     from events
                     where EventDate = '2019-05-15' 
                     group by DeviceID
                     having sum(events)>=1300)
group by EventDate
order by EventDate
"""
get_data(query)

,EventDate,events_cnt
0,2019-06-01,25
1,2019-06-02,12
2,2019-06-03,9
3,2019-06-04,3
4,2019-06-05,78
5,2019-06-06,499
6,2019-06-07,172
7,2019-06-08,211
8,2019-06-09,4
9,2019-06-10,9


#### Объединение таблиц - JOIN

Как мы узнали ранее, в реляционных базах данных таблицы имеют избыточные данные (ключи), для объединения таблиц друг с другом. И именно для объединения таблиц используется функция JOIN.

JOIN используется в блоке FROM, после первого источника. После JOIN указывается условие для объединения.

Базово, синтаксис выглядит так

    SELECT field 
    FROM table_one AS l 
    JOIN table_two AS r 
        ON l.key = r.key 
    

В данном примере мы указали первую таблицу как левую ( l ), вторую как правую ( r ), и указали, что они объединяются по ключу key.

Если с обеих сторон нашлось более одной строки с одинаковыми значениями, то по всем этим строкам строится декартово произведение (если явно не указано обратное).

Джойны бывают разных видов. В случае Clickhouse это:

* **INNER** (по умолчанию) — строки попадают в результат, только если значение ключевых колонок присутствует в обеих таблицах.
* **FULL**, **LEFT** и **RIGHT** — при отсутствии значения в обеих или в одной из таблиц включает строку в результат, но оставляет пустыми (NULL) колонки, соответствующие противоположной таблице.
* **CROSS** — декартово произведение двух таблиц целиком без указания ключевых колонок, секция с ON/USING явно не пишется;


<a> <img src="https://i.pinimg.com/originals/c7/07/f9/c707f9cdc08b1cdd773c006da976c8e6.jpg" width="800" height="160" ></a>

JOIN'ы могут иметь различную строгость. 
Перед JOIN'ом модет стоять модицицирующее выражение, например:

**ANY INNER JOIN**

**ALL** — если правая таблица содержит несколько подходящих строк, то ClickHouse выполняет их декартово произведение. Это стандартное поведение JOIN в SQL.

**ANY** — если в правой таблице несколько соответствующих строк, то присоединяется только первая найденная. Если в правой таблице есть только одна подходящая строка, то результаты ANY и ALL совпадают.

Чтобы посмотреть как вживую работает JOIN, давайте посмотрим, какие UserID совершили установки приложения. Для этого нужно взять таблицу Installs, выбрать из нее все поля и приджойнить ее по DeviceID к таблице devices. Чтобы на результат можно было посмотреть, выведем только 10 записей.

In [12]:
query = '''
select  a.Source as Source, 
    a.DeviceID as DeviceID,
    a.InstallCost as InstallCost,
    a.InstallationDate as InstallationDate,
    b.UserID as UserID
from installs as a
inner join devices as b 
on a.DeviceID = b.DeviceID
where a.InstallationDate between '2019-01-01' and '2019-06-30'
limit 10'''
get_data(query)

,Source,DeviceID,InstallCost,InstallationDate,UserID
0,Source_27,17916074153845327618,0,2019-01-15,12749113617023979255
1,Source_27,2758078266442941692,0,2019-06-17,13400567998165892481
2,Source_9,18443233034011339214,87,2019-01-09,4071437515845829412
3,Source_14,17762945135899196671,33,2019-05-24,5917907482760234754
4,Source_9,1447713209497065721,199,2019-02-01,5534547682854019833
5,Source_27,11988638464710050792,0,2019-01-23,3964087940576607028
6,Source_9,4733964167798139506,96,2019-02-06,15747807814772320889
7,Source_27,14417301148958195840,0,2019-02-27,2676958798372357803
8,Source_27,15892686439777271910,0,2019-04-04,2719448092992395789
9,Source_18,7585022962318633168,0,2019-05-25,6486963586472491159
